# WEB SCRAPING NI ASSEMBLY WEBSITE
Here we will create a web scraping model that not only extracts data from the current year, but also extracts data from the previous years as well.

Defining the URL:

In [1]:
url = 'http://aims.niassembly.gov.uk/officialreport/reports.aspx'

## DEPENDENCIES REQUIRED

In [2]:
pip install selenium

     |████████████████████████████████| 911kB 2.7MB/s 


In [3]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys    
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from tqdm import tqdm
import time

For using selenium explicitly we need to make some changes. 

This isn't required if we use a local or virtual machine to run our web-scraping model

In [4]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 35 not upgraded.
Need to get 75.5 MB of archives.
After this operation, 256 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 83.0.4103.61-0ubuntu0.18.04.1 [1,119 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 83.0.410

By inspecting the NI Assembly Hansard website we can see that each of the assembly sessions can be viewed in the website itself and also can be downloaded in pdf format. Lets view the page source of this website

![alt text](https://i.imgur.com/1s7qO7R.png)

Here we initialize a driver (virtual browser). This is done so that we are able to maneuver within the NI Assembly website. 

In [5]:
driver = webdriver.Chrome('chromedriver',options=options)
driver.get(url)
# print(driver.page_source) # uncomment to see the the whole page source of the website

We will create an empty DataFrame to save all the data extracted from our web scraping model.

In [6]:
ni_dataframe = pd.DataFrame(columns=['date', 'heading_1', 'heading_2', 'heading_3', 'contribution_minister_name', 'contribution', 'procedure', 'motion'], index = [0])
ni_dataframe

,date,heading_1,heading_2,heading_3,contribution_minister_name,contribution,procedure,motion
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Next we will extract the link address of all the NI assembly sessions that is available in a single page. This is stored in a form of list which can be later iterated using a for-loop to extract the the data (date, minister_name, his/her statements) from individual sessions.

In [7]:
# all_views = driver.find_elements_by_tag_name("a")
all_views = driver.find_elements_by_xpath('//*[@href]')

# print(all_views[197].get_attribute('innerHTML'))
# print(all_views[199].get_attribute("textContent"))
# # print(all_views[199].get_attribute('value'))
# # print(all_views[199].text)
print(all_views[200].get_attribute("href"))
print(all_views[200].get_attribute("id"))
print(len(all_views))

final_view_href = []

for i in range(len(all_views)):
  if all_views[i].get_attribute('textContent') == 'View':
    # print(all_views[i].get_attribute("href"))
    final_view_href.append(all_views[i].get_attribute("href"))
print("Number of NI Assembly session links available on the page: ",len(final_view_href))

# OR

final_view_id = []
for i in range(len(all_views)):
  if all_views[i].get_attribute('textContent') == 'View':
    # print(all_views[i].get_attribute("id"))
    final_view_id.append(all_views[i].get_attribute("id"))

# OR

final_view_full = []
for i in range(len(all_views)):
  if all_views[i].get_attribute('textContent') == 'View':
    # print(all_views[i])
    final_view_full.append(all_views[i])



"""We can iterate over the session reports in two ways.
1. By the href values that we filtered out which had 'View' in its contents. This
is then passed through a for loop and clicked into using a try/exception block system.
Then essential data is extracted.
2. By filtering out of href values and then getting the id of each. This is then passed through a 
clicker function where the virtual browser goes inside the session report page, then parse the
html to extract the required information.
The fastest method will be implemented. We will use tqdm for this purpose.
"""


print("Number of NI Assembly session links available on the page: ",len(final_view_id))

http://aims.niassembly.gov.uk/officialreport/report.aspx?&eveDate=2020/05/12&docID=300902
ctl00_MainContentPlaceHolder_OfficialReportsGridView_ctl13_HTMLViewButton1
292
Number of NI Assembly session links available on the page:  30
Number of NI Assembly session links available on the page:  30


We were able to extract 30 links from a single page. By manually cross-checking the links myself, this came out to be accurate. Thus, we will use this code-snippet for all the sub-pages as well as when we iterate through different years.

We can iterate over the session reports in two ways.
1. By the href values that we filtered out which had 'View' in its contents. This
is then passed through a for loop and clicked into using a try/exception block system.
Then essential data is extracted.
2. By filtering out of href values and then getting the id of each. This is then passed through a 
clicker function where the virtual browser goes inside the session report page, then parse the
html to extract the required information.

The fastest method will be implemented. We will use tqdm for this purpose.

In [8]:
# METHOD 1 (using href)
try:
    for i in tqdm(final_view_href, total=len(final_view_href)) :
      driver = webdriver.Chrome('chromedriver',options=options)
      driver.get(i)

      ni_text = driver.find_element_by_tag_name("main")
      # print(ni_text.text)

except:
    print("error!")
    driver.quit()

100%|██████████| 30/30 [06:09<00:00, 12.32s/it]


In [9]:
# METHOD 2 (using id and clicker)
driver = webdriver.Chrome('chromedriver',options=options)
driver.get(url)
try:
  for h in tqdm(final_view_id, total=len(final_view_id)):

    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, h))
        )
    element.click()
    ni_text = driver.find_element_by_tag_name("main")      # tag_name
    # ni_header = driver.find_elements_by_tag_name("p")
    # print(ni_header.text)
    driver.back()

except:
    print("error!")
    driver.quit()

100%|██████████| 30/30 [00:46<00:00,  1.55s/it]


We can see that method 2 takes significantly lesser time

Next step we will  try to extract specific parts of the corpus. For example lets try to filter out all the Headings for the whole 30 Hansard reports from a single page of the year 2019-2020.

In [10]:
# METHOD 2 (using id and clicker)- filtering out Header1
driver = webdriver.Chrome('chromedriver',options=options)
driver.get(url)
try:
  for h in tqdm(final_view_id, total=len(final_view_id)):

    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, h))
        )
    element.click()
    main = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "main"))
        )
    ni_header_1 = main.find_elements_by_class_name("Header1")
    ni_header_2 = main.find_elements_by_class_name("Header2")
    ni_header_3 = main.find_elements_by_class_name("Header3")
    print("X"*50, "HEADER 1", "X"*50)    
    for h in ni_header_1:
        print(h.text)
    print("X"*50, "HEADER 2", "X"*50)
    for i in ni_header_2:
        print(i.text)
    print("X"*50, "HEADER 3", "X"*50)
    for j in ni_header_3:
        print(j.text)
         
    driver.back()

except:
    print("error!")
    driver.quit()

  0%|          | 0/30 [00:00<?, ?it/s]

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX HEADER 1 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Assembly Business
Executive Committee Business
Committee Business
Question for Urgent Oral Answer
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX HEADER 2 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Agriculture, Environment and Rural Affairs
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX HEADER 3 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Standing Order 20(1): Suspension
Northern Ireland Assembly Commissioner for Standards: Appointment
Health Protection (Coronavirus, Restrictions) (Amendment No. 9) Regulations (Northern Ireland) 2020
Health Protection (Coronavirus, Restrictions) (Amendment No. 10) Regulations (Northern Ireland) 2020
Executive Committee (Functions) Bill: Consideration Stage
Climate Change and the Introduction of a Climate Change Act
Waste Storage at Edenderry Industrial Estate


  3%|▎         | 1/30 [00:01<00:42,  1.47s/it]

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX HEADER 1 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Assembly Business
Executive Committee Business
Committee Business
Oral Answers to Questions
Committee Business
Private Members' Business
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX HEADER 2 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
The Executive Office
Health
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX HEADER 3 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Standing Order 20A: Suspension
Pension Schemes Bill: Second Stage
Sea Fish Industry (Coronavirus) (Fixed Costs) Scheme (Northern Ireland) 2020
Sea Fish Industry (Coronavirus) (Fixed Costs) (Amendment) Scheme (Northern Ireland) 2020
The Rates (Exemption for Automatic Telling Machines in Rural Areas) Order (Northern Ireland) 2020
Business and Planning Bill: Legislative Consent Motion
COVID-19 Guidance and Financial Support to Industry Sectors
Victims' Payment Scheme
Hart Inquiry
Urban Villages: Nort

  7%|▋         | 2/30 [00:05<01:05,  2.32s/it]

...

 93%|█████████▎| 28/30 [00:53<00:04,  2.04s/it]

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX HEADER 1 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Assembly Business
Ministerial Statement
Executive Committee Business
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX HEADER 2 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX HEADER 3 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Standing Order 20(1): Suspension
Tributes to former deputy First Minister Séamus Mallon
Committee for Health: Deputy Chairperson
Assembly Commission: Appointments
Extension of Sitting
Public Expenditure: 2019-2020 January Monitoring Round
Direct Payments to Farmers (Legislative Continuity) Bill: Legislative Consent Motion


 97%|█████████▋| 29/30 [00:54<00:01,  1.85s/it]

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX HEADER 1 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Assembly Business
Executive Committee Business
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX HEADER 2 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX HEADER 3 XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Assembly Commission: Vacancies
Standing Order 20(1): Suspension
Business Committee Membership
Standing Order 49(2)(a) and Standing Order 52(2)(a): Suspension
Statutory Committee Membership
Standing Committee Membership
European Union (Withdrawal Agreement) Bill
European Union (Withdrawal Agreement) Bill: Motion to Delay
European Union (Withdrawal Agreement) Bill


100%|██████████| 30/30 [00:55<00:00,  1.86s/it]


In this step we will create an architecture model that extracts data in a specific Dataframe format which will help us for creating our KG as well as for future NLP analysis.

In this section we will figure out how our virtual browser will click through different pages in a same year to extract data.

In [11]:
driver = webdriver.Chrome('chromedriver',options=options)
driver.get(url)


class_pages = driver.find_element_by_class_name('paging')
all_pages = class_pages.find_elements_by_tag_name('a')
final_pages_id = []
for i in range(len(all_pages)):
  final_pages_id.append(all_pages[i].get_attribute("href"))


all_views = driver.find_elements_by_xpath('//*[@href]')     # EXTRACTING SESSION LINK OF A PAGE
final_view_id = []
for i in range(len(all_views)):
  if all_views[i].get_attribute('textContent') == 'View':
    final_view_id.append(all_views[i].get_attribute("id"))
print("Number of Session links on this page: ", len(final_view_id))


element = driver.find_element_by_xpath("//td/a[text()='2']").click()

WebDriverWait(driver, 10).until(EC.staleness_of(driver.find_element_by_xpath("//td/a[text()='2']")))

all_views = driver.find_elements_by_xpath('//*[@href]')     # EXTRACTING SESSION LINK OF A PAGE

final_view_id = []
for j in range(len(all_views)):
  if all_views[j].get_attribute('textContent') == 'View':
    final_view_id.append(all_views[j].get_attribute("id"))
print("Number of Session links on this page: ", len(final_view_id))

driver.quit()

Number of Session links on this page:  30
Number of Session links on this page:  3


In [12]:
# METHOD 2 (using id and clicker)- main
driver = webdriver.Chrome('chromedriver',options=options)
driver.get(url)
ni_dataframe = pd.DataFrame(columns=['date', 'heading_1', 'heading_2', 'heading_3', 'contribution_minister_name', 'contribution', 'procedure', 'motion'])

all_views = driver.find_elements_by_xpath('//*[@href]')     # EXTRACTING SESSION LINK OF A PAGE
final_view_id = []
for i in range(len(all_views)):
  if all_views[i].get_attribute('textContent') == 'View':
    final_view_id.append(all_views[i].get_attribute("id"))
print("Number of Session links on this page: ", len(final_view_id))

try:
  for h in tqdm(final_view_id, total=len(final_view_id)):

    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, h))
        )
    element.click()
    main = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "main"))
        )

    ni_date = main.find_element_by_tag_name("h1")
    new_row = {'date':'', 'heading_1':'', 'heading_2':'', 'heading_3':'', 'contribution_minister_name':'', 'contribution':'', 'procedure':'', 'motion':''}
    new_row['date'] = ni_date.text

    ni_div = main.find_elements_by_tag_name("div")
    
    for i in range(len(ni_div)):
      if ni_div[i].get_attribute('class') == 'Header1':
        new_row['heading_1'] = ni_div[i].text
      if ni_div[i].get_attribute('class') == 'Header2':
        new_row['heading_2'] = ni_div[i].text 
      if ni_div[i].get_attribute('class') == 'Header3':
        new_row['heading_3'] = ni_div[i].text
      if ni_div[i].get_attribute('class') == 'Procedure':
        new_row['procedure'] = new_row['procedure'] + ni_div[i].text +" "
      if ni_div[i].get_attribute('class') == 'Motion':
        new_row['motion'] = new_row['motion'] + ni_div[i].text +" "
      
      if ni_div[i].get_attribute('class') == 'Contribution':
        ni_name_and_contribution = ni_div[i].text
        if (ni_name_and_contribution.find(':') != -1):
          ni_name, ni_contribution = ni_name_and_contribution.split(":", 1)
          new_row['contribution_minister_name'] = ni_name
          new_row['contribution'] = ni_contribution

        else:
          new_row['contribution_minister_name'] = ni_name
          new_row['contribution'] = ni_name_and_contribution

        ni_dataframe = ni_dataframe.append(new_row, ignore_index=True)

        new_row['procedure'] = ''
        new_row['motion'] = ''
          
    driver.back()

except:
    print("error!")
    driver.quit()

  0%|          | 0/30 [00:00<?, ?it/s]

Number of Session links on this page:  30


100%|██████████| 30/30 [12:25<00:00, 24.85s/it]


In [13]:
ni_dataframe.head(10)

,date,heading_1,heading_2,heading_3,contribution_minister_name,contribution,procedure,motion
0,Official Report: Tuesday 21 July 2020,Assembly Business,,,Mr Deputy Speaker (Mr Beggs),"Before the first item of business, I remind M...",,
1,Official Report: Tuesday 21 July 2020,Assembly Business,,Standing Order 20(1): Suspension,Mr Butler,I beg to move,,
2,Official Report: Tuesday 21 July 2020,Assembly Business,,Standing Order 20(1): Suspension,Mr Deputy Speaker (Mr Beggs),"Before we proceed to the Question, I remind M...",,That Standing Order 20(1) be suspended for 21 ...
3,Official Report: Tuesday 21 July 2020,Executive Committee Business,,"Health Protection (Coronavirus, Restrictions) ...",Mr Deputy Speaker (Mr Beggs),The next two motions are to approve statutory...,Question put and agreed to.\n\nResolved (with ...,That Standing Order 20(1) be suspended for 21 ...
4,Official Report: Tuesday 21 July 2020,Executive Committee Business,,"Health Protection (Coronavirus, Restrictions) ...","Mr Kearney (Junior Minister, The Executive Off...",Éirím leis an rún a chur chun cinn. I beg to ...,,
5,Official Report: Tuesday 21 July 2020,Executive Committee Business,,"Health Protection (Coronavirus, Restrictions) ...",Mr Deputy Speaker (Mr Beggs),The Business Committee has agreed that there ...,,"That the Health Protection (Coronavirus, Restr..."
6,Official Report: Tuesday 21 July 2020,Executive Committee Business,,"Health Protection (Coronavirus, Restrictions) ...",Mr Kearney,There are two motions before the Assembly tod...,,
7,Official Report: Tuesday 21 July 2020,Executive Committee Business,,"Health Protection (Coronavirus, Restrictions) ...",Mr Kearney,"Regrettably, other places have not experienced...",,
8,Official Report: Tuesday 21 July 2020,Executive Committee Business,,"Health Protection (Coronavirus, Restrictions) ...",Mr McGrath (The Chairperson of the Committee f...,I am speaking on behalf of the Committee for ...,,
9,Official Report: Tuesday 21 July 2020,Executive Committee Business,,"Health Protection (Coronavirus, Restrictions) ...",Mr McGrath (The Chairperson of the Committee f...,I will now make a few remarks as an SDLP repre...,,


In [14]:
ni_dataframe.shape

(8996, 8)

## XXXXXXX WORK IN PROGRESS XXXXXXX

Since all of the NI assembly sessions in a particular  year cannot be contained in a single html page NI assembly website shows them in several other pages that we can navigate through. Our Web scraping model should go to these pages and perform the same data extraction technique that we developed previously.

In [16]:
# METHOD 2 (using id and clicker)- main
driver = webdriver.Chrome('chromedriver',options=options)
driver.get(url)
ni_dataframe = pd.DataFrame(columns=['date', 'heading_1', 'heading_2', 'heading_3', 'contribution_minister_name', 'contribution', 'procedure', 'motion'])

#### WORK SPACE FOR SCROLLABLE YEARS


#### WORK SPACE FOR SCROLLABLE YEARS

# We get the number of pages in a particular year (if any)
class_pages = driver.find_element_by_class_name('paging')  
all_pages = class_pages.find_elements_by_tag_name('a')
final_pages_id = []
for h in range(len(all_pages)):
  final_pages_id.append(all_pages[h].get_attribute("href"))
print(len(all_pages))

# MAIN
try:
  for i in range(len(final_pages_id)+1):     # LOOPING THROUGH PAGES

    all_views = driver.find_elements_by_xpath('//*[@href]')     # EXTRACTING ASSEMBLY SESSION LINKS OF A PAGE
    final_view_id = []
    for j in range(len(all_views)):
      if all_views[j].get_attribute('textContent') == 'View':
        final_view_id.append(all_views[j].get_attribute("id"))
        print(all_views[j].get_attribute("href"))
    print("Number of Session links on this page: ", len(final_view_id))
    
    for k in tqdm(final_view_id, total=len(final_view_id)):     # LOOPING THROUGH SESSION LINKS
      element = WebDriverWait(driver, 10).until(
          EC.presence_of_element_located((By.ID, k))
          )
      print(element.get_attribute("href"))
      driver.execute_script("arguments[0].click();", element)
      main = WebDriverWait(driver, 10).until(
          EC.presence_of_element_located((By.TAG_NAME, "main"))
          )

      ni_date = main.find_element_by_tag_name("h1")
      new_row = {'date':'', 'heading_1':'', 'heading_2':'', 'heading_3':'', 'contribution_minister_name':'', 'contribution':'','quote':'', 'procedure':'', 'motion':''}
      new_row['date'] = ni_date.text

      ni_div = main.find_elements_by_tag_name("div")
      print(len(ni_div))
      
      for m in range(len(ni_div)):
        if ni_div[m].get_attribute('class') == 'Header1':
          new_row['heading_1'] = ni_div[m].text
        if ni_div[m].get_attribute('class') == 'Header2':
          new_row['heading_2'] = ni_div[m].text 
        if ni_div[m].get_attribute('class') == 'Header3':
          new_row['heading_3'] = ni_div[m].text
        if ni_div[m].get_attribute('class') == 'Procedure':
          new_row['procedure'] = new_row['procedure'] + ni_div[m].text +" "
        if ni_div[m].get_attribute('class') == 'Motion':
          new_row['motion'] = new_row['motion'] + ni_div[m].text +" "
        if ni_div[m].get_attribute('class') == 'Quote':
          new_row['quote'] = ni_div[m].text 
        
        if ni_div[m].get_attribute('class') == 'Contribution':
          ni_name_and_contribution = ni_div[m].text
          if (ni_name_and_contribution.find(':') != -1):
            ni_name, ni_contribution = ni_name_and_contribution.split(":", 1)
            new_row['contribution_minister_name'] = ni_name
            new_row['contribution'] = ni_contribution.rstrip()

          else:
            new_row['contribution_minister_name'] = ni_name
            new_row['contribution'] = ni_name_and_contribution.rstrip()

          ni_dataframe = ni_dataframe.append(new_row, ignore_index=True)

          new_row['procedure'] = ''
          new_row['motion'] = ''
          new_row['quote'] = ''
            
      driver.back()
      if i != 0:
        element_1 = driver.find_element_by_xpath(f"//td/a[text()='2']").click()
        WebDriverWait(driver, 10).until(EC.staleness_of(driver.find_element_by_xpath("//td/a[text()='2']")))
    if i != len(final_pages_id):
      element_1 = driver.find_element_by_xpath(f"//td/a[text()='2']").click()
      WebDriverWait(driver, 10).until(EC.staleness_of(driver.find_element_by_xpath("//td/a[text()='2']")))
except:
    print("error!")
    driver.quit()

1
http://aims.niassembly.gov.uk/officialreport/report.aspx?&eveDate=2020/07/21&docID=304884
http://aims.niassembly.gov.uk/officialreport/report.aspx?&eveDate=2020/07/07&docID=304152
http://aims.niassembly.gov.uk/officialreport/report.aspx?&eveDate=2020/07/06&docID=304151
http://aims.niassembly.gov.uk/officialreport/report.aspx?&eveDate=2020/06/30&docID=303726
http://aims.niassembly.gov.uk/officialreport/report.aspx?&eveDate=2020/06/23&docID=302713
http://aims.niassembly.gov.uk/officialreport/report.aspx?&eveDate=2020/06/16&docID=302204
http://aims.niassembly.gov.uk/officialreport/report.aspx?&eveDate=2020/06/09&docID=301801
http://aims.niassembly.gov.uk/officialreport/report.aspx?&eveDate=2020/06/02&docID=301413
http://aims.niassembly.gov.uk/officialreport/report.aspx?&eveDate=2020/06/01&docID=301412
http://aims.niassembly.gov.uk/officialreport/report.aspx?&eveDate=2020/05/26&docID=301336
http://aims.niassembly.gov.uk/officialreport/report.aspx?&eveDate=2020/05/19&docID=301084
http://a

  0%|          | 0/30 [00:00<?, ?it/s]

http://aims.niassembly.gov.uk/officialreport/report.aspx?&eveDate=2020/07/21&docID=304884
248


  3%|▎         | 1/30 [00:14<06:51, 14.18s/it]

http://aims.niassembly.gov.uk/officialreport/report.aspx?&eveDate=2020/07/07&docID=304152
714


...

 33%|███▎      | 1/3 [00:25<00:51, 25.74s/it]

http://aims.niassembly.gov.uk/officialreport/report.aspx?&eveDate=2020/01/11&docID=290745
219


 67%|██████▋   | 2/3 [00:40<00:22, 22.48s/it]

http://aims.niassembly.gov.uk/officialreport/report.aspx?&eveDate=2019/10/21&docID=290622
101


100%|██████████| 3/3 [00:48<00:00, 16.18s/it]


In [17]:
ni_dataframe.to_csv(r'/content/drive/My Drive/Colab Notebooks/QARIK_placement_project/web_scraping/output/ni_assembly_hansard.csv', index = False)

In [18]:
ni_dataframe.shape

(9522, 9)

In [19]:
""" THINGS TO DO & CORRECT:
1) Very few contribution blocks are partially extracted. This is due to the 'new line' code present and our model ignores the rest text when it detects a "\n.
We need to figure out a way to combine paragraphs seperated by a line. (tried rstrip() to remove the "\n", didn't change the outcome)
2) Procedure, motion, quotes appear in one cell after it should originally be located. ( This issue doesn't effect our goal but will correct this)
3) The page number in the javascript clicker should be replaced by the for loop number (i) instead of being manually written.
4) Scrolling through year"""

' THINGS TO DO & CORRECT:\n1) Very few contribution blocks are partially extracted. This is due to the \'new line\' code present and our model ignores the rest text when it detects a "\n.\nWe need to figure out a way to combine paragraphs seperated by a line. (tried rstrip() to remove the "\n", didn\'t change the outcome)\n2) Procedure, motion, quotes appear in one cell after it should originally be located. ( This issue doesn\'t effect our goal but will correct this)\n3) The page number in the javascript clicker should be replaced by the for loop number (i) instead of being manually written.\n4) Scrolling through year'